# DeepAR - Experiments

### DeepAR (Salinas et al., 2017)

The **DeepAR** algorithm uses autoregressive features that are used as the inputs to a recurrent neural network. The RNN uses LSTM cells as default but can also changed to GRU cells. In both cases previous time points are taken as input. 

- [`DeepAREestimator`](https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html)

In [1]:
import gluonts
gluonts.__version__

'0.4.0'

In [2]:
# !pip install gluonts

In [3]:
# standard imports
import numpy as np
import pandas as pd

# json
import json

# gluon data
from gluonts.dataset.repository.datasets import get_dataset, dataset_recipes
from gluonts.dataset.util import to_pandas

# gluon imports
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator

# model imports
# from gluonts.model.deepstate import DeepStateEstimator
# from gluonts.model.deep_factor import DeepFactorEstimator
from gluonts.model.deepar import DeepAREstimator

import mxnet as mx
from pprint import pprint


INFO:root:Using CPU


In [4]:
def deepar(data="m4_quarterly", seed=42, epochs=100, batches=50):
    
    dataset = get_dataset(data, regenerate=False)
    mx.random.seed(seed)
    np.random.seed(seed)
    
    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=epochs,      
        num_batches_per_epoch=batches,      
        learning_rate=1e-3,
    )
    
    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
        cardinality=[cardinality],
        context_length=dataset.metadata.prediction_length,
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
        use_feat_static_cat=True   
    )
        
    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_samples=100
    )
    
    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )
    
    metrics = ["MASE", "sMAPE", "MSIS", "wQuantileLoss[0.5]", "wQuantileLoss[0.9]"]
    output = {key: round(value, 8) for key, value in agg_metrics.items() if key in metrics}
    output["epochs"] = epochs
    output["seed"] = seed
    return(output)


In [5]:
%%time
res = deepar(data="m4_quarterly", seed=42, epochs=1000, batches=200)
pprint(res)

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_quarterly.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 200/200 [00:28<00:00,  7.04it/s, avg_epoch_loss=7.68]
INFO:root:Epoch[0] Elapsed time 28.429 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.682600
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 200/200 [00:26<00:00,  7.59it/s, avg_epoch_loss=7.19]
INFO:root:Epoch[1] Elapsed time 26.355 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=7.187987
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 200/200 [00:19<00:00, 10.29it/s, avg_epoch_loss=7.21]
INFO:root:Epoch[2] Elapsed time 19.456 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=7.209310
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 200/200 [00:18<00:00, 10.81it/s, avg_epoch_loss=7.11]
INFO:root:Epoch[3] Elapsed time 18.512 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=7

INFO:root:Epoch[35] Evaluation metric 'epoch_loss'=6.575915
INFO:root:Epoch[36] Learning rate is 0.001
100%|██████████| 200/200 [00:19<00:00, 10.22it/s, avg_epoch_loss=6.57]
INFO:root:Epoch[36] Elapsed time 19.578 seconds
INFO:root:Epoch[36] Evaluation metric 'epoch_loss'=6.570769
INFO:root:Epoch[37] Learning rate is 0.001
100%|██████████| 200/200 [00:20<00:00, 10.00it/s, avg_epoch_loss=6.49]
INFO:root:Epoch[37] Elapsed time 20.011 seconds
INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=6.487315
INFO:root:Epoch[38] Learning rate is 0.001
100%|██████████| 200/200 [00:19<00:00, 10.18it/s, avg_epoch_loss=6.44]
INFO:root:Epoch[38] Elapsed time 19.662 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=6.439981
INFO:root:Epoch[39] Learning rate is 0.001
100%|██████████| 200/200 [00:20<00:00,  9.88it/s, avg_epoch_loss=6.45]
INFO:root:Epoch[39] Elapsed time 20.248 seconds
INFO:root:Epoch[39] Evaluation metric 'epoch_loss'=6.454008
INFO:root:Epoch[40] Learning rate is 0.001
100%|████

INFO:root:Epoch[72] Learning rate is 0.00025
100%|██████████| 200/200 [00:19<00:00, 10.34it/s, avg_epoch_loss=6.26]
INFO:root:Epoch[72] Elapsed time 19.351 seconds
INFO:root:Epoch[72] Evaluation metric 'epoch_loss'=6.261677
INFO:root:Epoch[73] Learning rate is 0.00025
100%|██████████| 200/200 [00:19<00:00, 10.31it/s, avg_epoch_loss=6.37]
INFO:root:Epoch[73] Elapsed time 19.398 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=6.373164
INFO:root:Epoch[74] Learning rate is 0.00025
100%|██████████| 200/200 [00:19<00:00, 10.17it/s, avg_epoch_loss=6.45]
INFO:root:Epoch[74] Elapsed time 19.676 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=6.446017
INFO:root:Epoch[75] Learning rate is 0.00025
100%|██████████| 200/200 [00:22<00:00,  9.06it/s, avg_epoch_loss=6.36]
INFO:root:Epoch[75] Elapsed time 22.073 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=6.358798
INFO:root:Epoch[76] Learning rate is 0.00025
100%|██████████| 200/200 [00:25<00:00,  7.84it/s, avg_epoch

KeyboardInterrupt: 

In [ ]:
%%time
res = deepar(data="m4_yearly", seed=42, epochs=1000, batches=200)
pprint(res)

In [ ]:
!jupyter nbconvert --output-dir="./html_outputs" --to html "DeepAR 09 - DeepAR experiments with gluonts".ipynb

In [ ]:
print("Done!")

In [12]:
# %%time
# results = []

# if __name__ == "__main__":
#     for i in range(42, 45):
#         print("Seed:", i)
#         res = deepar(data="m4_monthly", seed=i, epochs=100, batches=1000)
#         pprint(res)
#         results.append(res)
    
# results